In [1]:
import numpy as np
import pandas as pd
app_energy = pd.read_csv(r"~\Desktop\HAMOYE\STAGE B\Energy Data.csv")

In [2]:
app_energy = app_energy.rename(columns = {"T1": "Kitchen temp", "RH_1": "Kitchen humd", "T2": "Livingroom temp",
                                         "RH_2": "Livingroom humd", "T3": "Laundry temp", "RH_3": "Laundry humd",
                                         "T4": "Office temp", "RH_4": "Office humd", "T5": "Bathroom temp", "RH_5": "Bathroom humd",
                                          "T6": "Outside temp(North)", "RH_6": "Outside humd(North)", "T7": "Ironingroom temp",
                                         "RH_7": "Ironingroom humd", "T8": "Teenageroom temp", "RH_8": "Teenageroom humd",
                                         "T9": "Parentroom temp", "RH_9": "Parentroom humd", "T_out": "Baselinetemp outside", 
                                         "Press_mm_hg": "baselinepressure", "RH_out": "Baselinepressure outside",
                                         "Windspeed": "Windspeed(m/s)", "Visibility": "Visibility(km)", "Tdewpoint": "Dew point temp",
                                         "rv1": "RV1", "rv2": "RV2"})

In [4]:
app_energy = app_energy.drop(["date", "lights"], axis = 1)

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(app_energy), columns = app_energy.columns)

In [35]:
x = normalised_df[["Livingroom temp"]]
y = normalised_df[["Outside temp(North)"]]

In [36]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

In [37]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)
predicted_values = linear_model.predict(x_test)
predicted_values

array([[0.23928945],
       [0.46794238],
       [0.23108472],
       ...,
       [0.3001772 ],
       [0.4297256 ],
       [0.3217686 ]])

In [38]:
#Question 12
from sklearn.metrics import r2_score
r2 = r2_score(y_test, predicted_values)
round(r2, 2)

0.64

In [39]:
X = normalised_df.drop(columns = ["Appliances"])
Y = normalised_df["Appliances"]

In [40]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = (train_test_split(X,Y, test_size = 0.3, random_state = 42))

In [41]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)
predicted_values = linear_model.predict(x_test)
predicted_values

array([0.03319482, 0.24412217, 0.0340092 , ..., 0.06843864, 0.09999407,
       0.05760656])

In [42]:
#Question 13
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
mae
round(mae, 2)

0.05

In [43]:
#Question 14
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 2)

45.35

In [45]:
#Question 15
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

0.088

In [44]:
#Question 16
from sklearn.metrics import r2_score
r2 = r2_score(y_test, predicted_values)
round(r2, 2)

0.15

In [19]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha = 0.5)
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [20]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [21]:
#comparing the effect of regularization
def get_weights_df(linear_model, feat, col_name):
    #this function returns the weight of every feature
    weights = pd.Series(linear_model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ["Features", col_name]
    weights_df[col_name].round(3)
    return weights_df

In [22]:
#Question 17 and 19
linear_model_weights = get_weights_df(linear_model, x_train, "Linear_Model_Weight")
ridge_weights_df = get_weights_df(ridge_reg, x_train, "Ridge_Weight")
lasso_weights_df = get_weights_df(lasso_reg, x_train, "Lasso_Weight")
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on = "Features")
final_weights = pd.merge(final_weights, lasso_weights_df, on = "Features")
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_Weight
0,Livingroom humd,-0.456662,-0.401113,-0.000000
1,Baselinetemp outside,-0.321969,-0.250943,0.000000
2,Livingroom temp,-0.236198,-0.193899,0.000000
3,Parentroom temp,-0.189977,-0.188615,-0.000000
4,Teenageroom humd,-0.157585,-0.156584,-0.000110
5,Baselinepressure outside,-0.077748,-0.050642,-0.049557
6,Ironingroom humd,-0.044620,-0.046297,-0.000000
7,Parentroom humd,-0.039808,-0.041707,-0.000000
8,Bathroom temp,-0.015645,-0.020714,-0.000000
9,Kitchen temp,-0.003242,-0.021515,0.000000


In [27]:
#Question 18
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha = 0.5)
ridge_reg.fit(x_train, y_train)
predicted_value = ridge_reg.predict(x_test)

In [28]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_value))
round(rmse, 3)

0.088

In [33]:
#Question 20
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)
predicted_valuess = lasso_reg.predict(x_test)

In [34]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_valuess))
round(rmse, 3)

0.094